In [1]:
import joblib
import pandas as pd

In [2]:
loaded_model = joblib.load("CatBoost_model_basic.joblib")

In [15]:
data = pd.read_parquet("data/final_test.parquet")

In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51440 entries, 0 to 51439
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   counter_id                 51440 non-null  category      
 1   counter_name               51440 non-null  category      
 2   site_id                    51440 non-null  int64         
 3   site_name                  51440 non-null  category      
 4   date                       51440 non-null  datetime64[ns]
 5   counter_installation_date  51440 non-null  datetime64[ns]
 6   coordinates                51440 non-null  category      
 7   counter_technical_id       51440 non-null  category      
 8   latitude                   51440 non-null  float64       
 9   longitude                  51440 non-null  float64       
dtypes: category(5), datetime64[ns](2), float64(2), int64(1)
memory usage: 2.2 MB


In [17]:
data["date"] = pd.to_datetime(data["date"])
data["day_of_week"] = data["date"].dt.dayofweek
data["month"] = data["date"].dt.month
data["hour"] = data["date"].dt.hour

In [18]:
categorical_cols = [
    "counter_name",
    "site_name",
    "counter_technical_id",
    "day_of_week",
    "month",
    "hour",
]
numerical_cols = ["latitude", "longitude"]

In [19]:
for feature in categorical_cols:
    data[feature] = data[feature].astype(str)

In [20]:
cols_to_drop = [
    "counter_id",
    "site_id",
    "date",
    "counter_installation_date",
    "coordinates",
]

In [21]:
data = data.drop(cols_to_drop, axis=1)

In [22]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51440 entries, 0 to 51439
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   counter_name          51440 non-null  object 
 1   site_name             51440 non-null  object 
 2   counter_technical_id  51440 non-null  object 
 3   latitude              51440 non-null  float64
 4   longitude             51440 non-null  float64
 5   day_of_week           51440 non-null  object 
 6   month                 51440 non-null  object 
 7   hour                  51440 non-null  object 
dtypes: float64(2), object(6)
memory usage: 3.1+ MB


In [23]:
predictions = loaded_model.predict(data)

In [24]:
predictions_df = pd.DataFrame({"log_bike_count": predictions})

In [25]:
predictions_df.to_csv("predictions7.csv", index=True, index_label="Id")